In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['HOST_IP'] = '18.207.230.116'
os.environ['LOCAL_DB_USER'] = 'root'
os.environ['LOCAL_DB_PASSWORD'] = 'AlgoTeam123'
os.environ['LOCAL_DB_PORT'] = '3306'

tenant_id = 'test'

db_config = {
    'host': os.environ['HOST_IP'],
    'user': os.environ['LOCAL_DB_USER'],
    'password': os.environ['LOCAL_DB_PASSWORD'],
    'port': os.environ['LOCAL_DB_PORT']
}

In [ ]:
# %load apply_business_rule.py
# comment below two for local testing
# from ace_logger import Logging
# logging = Logging()
import pandas as pd
import ntpath

# uncomment these below lines for local testing
import logging 
logger=logging.getLogger() 
logger.setLevel(logging.DEBUG) 



import json
import os
from db_utils import DB 

from BusinessRules import BusinessRules

# one configuration
db_config = {
    'host': os.environ['HOST_IP'],
    'user': os.environ['LOCAL_DB_USER'],
    'password': os.environ['LOCAL_DB_PASSWORD'],
    'port': os.environ['LOCAL_DB_PORT']
}


# give any path...platform independent...get the base name
def path_leaf(path):
    """give any path...platform independent...get the base name"""
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)


def to_DT_data(parameters):
    """Amith's processing for parameters"""
    output = []
    try:
        for param_dict in parameters:
            print(param_dict)    
            if param_dict['column'] == 'Add_on_Table':
                output.append({'table': param_dict['table'],'column': param_dict['column'],'value': param_dict['value']})
                # Need to add a function to show this or tell Kamal check if its addon table and parse accordingly
            else:                
                output.append({'table': param_dict['table'],'column': param_dict['column'],'value': param_dict['value']})
    except:
        print("Error in to_DT_data()")
        traceback.print_exc()
        return []
    try:
        output = [dict(t) for t in {tuple(d.items()) for d in output}]
    except:
        print("Error in removing duplicate dictionaries in list")
        traceback.print_exc()
        pass
    return output

def get_data_sources(tenant_id, case_id, column_name, master=False):
    """Helper to get all the required table data for the businesss rules to apply
    """
    get_datasources_query = "SELECT * from `data_sources`"
    business_rules_db = DB('business_rules', tenant_id=tenant_id, **db_config)
    data_sources = business_rules_db.execute(get_datasources_query)


    # sources
    sources = json.loads(list(data_sources[column_name])[0])
    
    
    data = {}
    for database, tables in sources.items():
        db = DB(database, tenant_id=tenant_id, **db_config)
        for table in tables:
            if master:
                query = f"SELECT * from `{table}`"
                df = db.execute(query)
            else:
                query = f"SELECT * from `{table}` WHERE case_id = %s"
                params = [case_id]
                df = db.execute(query, params=params)
            if not df.empty:
                data[table] = df.to_dict(orient='records')[0]
            else:
                data[table] = {}
    
    
    case_id_based_sources = json.loads(list(data_sources['case_id_based'])[0])
    
    return data
                
def get_rules(tenant_id, group):
    """Get the rules based on the stage, tenant_id"""
    business_rules_db = DB('business_rules', tenant_id=tenant_id, **db_config)
    get_rules_query = "SELECT * from `sequence_rule_data` where `group` = %s"
    params = [group]
    rules = business_rules_db.execute(get_rules_query, params=params)
    return rules

def update_tables(case_id, tenant_id, updates):
    """Update the values in the database"""
    try:
        extraction_db = DB('extraction', tenant_id=tenant_id, **db_config) # only in ocr or process_queue we are updating
        queue_db = DB('queues', tenant_id=tenant_id, **db_config) # only in ocr or process_queue we are updating
        
        for table, colum_values in updates.items():
            if table == 'ocr':
                extraction_db.update(table, update=colum_values, where={'case_id':case_id})
            if table == 'process_queue':
                queue_db.update(table, update=colum_values, where={'case_id':case_id})
    except Exception as e:
        logging.error("Error in updating the tables")
        logging.error("check whether the table exists")
        logging.error(e)
        
    return "UPDATED IN THE DATABASE SUCCESSFULLY"


def writeToCsv(df, file_path, required_standard_mapping=None):
    """Write the dataframe to csv"""
    if not required_standard_mapping:
        required_standard_mapping = {ele:ele for ele in list(df.columns)}
    required_columns = required_standard_mapping.values()
    try:

        # first find whether any csv exists with the file_name
        existing_df = pd.read_csv(file_path)
        mode = 'a'
        headers = False
        df.to_csv(file_path, columns=required_columns, mode=mode, header=headers, index=False)
    except:
        # no sample file exists
        headers = True
        mode = 'w'
        
        # rename the columns in the dataframe to standardized columns            
        df = df.rename(columns=required_standard_mapping)
        df.to_csv(file_path, columns=required_columns, mode=mode, header=headers, index=False)
                
    return "Written to csv successfully"




# as of now run this...you can combine the run_chained_rules and column_chained rules with small changes
def run_chained_rules_column(file_path, chain_rules, start_rule_id=None):
    """Execute the chained rules column wise"""
    
    # get the mapping of the rules...basically a rule_id maps to a rule
    rule_id_mapping = {}
    for ind, rule in chain_rules.iterrows():
        rule_id_mapping[rule['rule_id']] = [rule['rule_string'], rule['next_if_sucess'], rule['next_if_failure'], rule['stage'], rule['description'], rule['data_source']]
    logging.info(f"\n rule id mapping is \n{rule_id_mapping}\n")
    
    # evaluate the rules one by one as chained
    # start_rule_id = None
    if start_rule_id is None:
        if rule_id_mapping.keys():
            start_rule_id = list(rule_id_mapping.keys())[0]
    
    BR  = BusinessRules(None, [], {})
    file_name = path_leaf(file_path)[:-4] # stripping the .csv
    BR.data_source[file_name] = pd.read_csv(file_path)
    
    logging.info(f"\nStart rule id got is {start_rule_id}\n ")
    while start_rule_id != "END":
        # get the rules, next rule id to be evaluated
        rule_to_evaluate, next_if_sucess, next_if_failure, stage, description, data_source = rule_id_mapping[str(start_rule_id)]  
    
        logging.info(f"\nInside the loop \n rule_to_evaluate  {rule_to_evaluate}\n \
                      \nnext_if_sucess {next_if_sucess}\n \
                      \nnext_if_failure {next_if_failure}\n ")
        
        # evaluate the rule
        BR.rules = [json.loads(rule_to_evaluate)] 
        BR.evaluate_business_rules() # apply the business rules
        start_rule_id = next_if_sucess # no matter what go with next if success
        logging.info(f"\n next rule id to execute is {start_rule_id}\n")
        
    logging.info("\n Applied chained rules successfully")
    # finally write to the csv
    writeToCsv(BR.data_source['file_name'])
    return BR



def run_chained_rules(case_id, tenant_id, chain_rules, start_rule_id=None, updated_tables=False, trace_exec=None, rule_params=None):
    """Execute the chained rules"""
    
    # get the mapping of the rules...basically a rule_id maps to a rule
    rule_id_mapping = {}
    for ind, rule in chain_rules.iterrows():
        rule_id_mapping[rule['rule_id']] = [rule['rule_string'], rule['next_if_sucess'], rule['next_if_failure'], rule['stage'], rule['description'], rule['data_source']]
    logging.info(f"\n rule id mapping is \n{rule_id_mapping}\n")
    
    # evaluate the rules one by one as chained
    # start_rule_id = None
    if start_rule_id is None:
        if rule_id_mapping.keys():
            start_rule_id = list(rule_id_mapping.keys())[0]
            trace_exec = []
            rule_params = {}
            
    # if start_rule_id then. coming from other service 
    # get the existing trace and rule params data
    business_rules_db = DB('business_rules', tenant_id=tenant_id, **db_config)
    rule_data_query = "SELECT * from `rule_data` where `case_id`=%s"
    params = [case_id]
    df = business_rules_db.execute(rule_data_query, params=params)
    try:
        trace_exec = json.loads(list(df['trace_data'])[0])
        logging.info(f"\nexistig trace exec is \n{trace_exec}\n")
    except Exception as e:
        logging.info(f"no existing trace data")
        logging.info(f"{str(e)}")
        trace_exec = []
    
    try:
        rule_params = json.loads(list(df['rule_params'])[0])
        logging.info(f"\nexistig rule_params is \n{rule_params}\n")
    except Exception as e:
        logging.info(f"no existing rule params data")
        logging.info(f"{str(e)}")
        rule_params = {}
       
    logging.info(f"\nStart rule id got is {start_rule_id}\n ")
    while start_rule_id != "END":
        # get the rules, next rule id to be evaluated
        rule_to_evaluate, next_if_sucess, next_if_failure, stage, description, data_source = rule_id_mapping[str(start_rule_id)]  
    
        logging.info(f"\nInside the loop \n rule_to_evaluate  {rule_to_evaluate}\n \
                      \nnext_if_sucess {next_if_sucess}\n \
                      \nnext_if_failure {next_if_failure}\n ")
        
        # update the data_table if there is any change
        case_id_data_tables = get_data_sources(tenant_id, case_id, 'case_id_based')
        master_updated_tables = {} 
        if updated_tables:
            master_updated_tables = get_data_sources(tenant_id, case_id, 'updated_tables')
        # consolidate the data into data_tables
        data_tables = {**case_id_data_tables, **master_data_tables, **master_updated_tables} 
        
        # evaluate the rule
        rules = [json.loads(rule_to_evaluate)] 
        BR  = BusinessRules(case_id, rules, data_tables)
        decision = BR.evaluate_rule(rules[0])
        
        logging.info(f"\n got the decision {decision} for the rule id {start_rule_id}")
        logging.info(f"\n updates got are {BR.changed_fields}")
        
        updates = {}
        # update the updates if any
        if BR.changed_fields:
            updates = BR.changed_fields
            update_tables(case_id, tenant_id, updates)

        
        # update the trace_data
        trace_exec.append(start_rule_id)

        logging.info(f"\n params data used from the rules are \n {BR.params_data}\n")
        # update the rule_params
        trace_dict = {
                        str(start_rule_id):{
                            'description' : description if description else 'No description available in the database',
                            'output' : "",
                            'input' : to_DT_data(BR.params_data['input'])
                            }
                        }
        rule_params.update(trace_dict)
        # update the start_rule_id based on the decision
        if decision:
            start_rule_id = next_if_sucess
        else:
            start_rule_id = next_if_failure
        logging.info(f"\n next rule id to execute is {start_rule_id}\n")
        
    
    # off by one updates...
    trace_exec.append(start_rule_id)
    
    # store the trace_exec and rule_params in the database
    update_rule_params_query = f"INSERT INTO `rule_data`(`id`, `case_id`, `rule_params`) VALUES ('NULL',%s,%s) ON DUPLICATE KEY UPDATE `rule_params`=%s"
    params = [case_id, json.dumps(rule_params), json.dumps(rule_params)]
    business_rules_db.execute(update_rule_params_query, params=params)
    
    update_trace_exec_query = f"INSERT INTO `rule_data` (`id`, `case_id`, `trace_data`) VALUES ('NULL',%s,%s) ON DUPLICATE KEY UPDATE `trace_data`=%s"
    params = [case_id, json.dumps(trace_exec), json.dumps(trace_exec)]
    business_rules_db.execute(update_trace_exec_query, params=params)
    
    logging.info("\n Applied chained rules successfully")
    return 'Applied chained rules successfully'

def run_group_rules(case_id, rules, data):
    """Run the rules"""
    rules = [json.loads(rule) for rule in rules] 
    BR  = BusinessRules(case_id, rules, data)
    updates = BR.evaluate_business_rules()
    logging.info(f"\n updates from the group rules are \n{updates}\n")
    return updates

def apply_business_rule(case_id, function_params, tenant_id, file_path):
    """Run the business rules based on the stage in function params and tenant_id
    Args:
        case_id: Unique id that we pass
        function_params: Parameters that we get from the configurations
        tenant_id: Tenant on which we have to apply the rules
    Returns:
    """
    updates = {} # keep a track of updates that are being made by business rules
    try:
        # get the stage from the function_parameters...As of now its first ele..
        # need to make generic or key-value pairs
        logging.info(f"\n case_id {case_id} \nfunction_params {function_params} \ntenant_id {tenant_id}\n")
        stage = function_params['stage'][0]
        
        # no case id passed meaning the operations we are doing on the column wise not case_id wise .
        # feature developed for karvy
        column = False
        if not case_id:
            column = True
            
        # get the rules
        rules = get_rules(tenant_id, stage)
        
        # get the mapping of the rules...basically a rule_id maps to a rule.
        # useful for the chain rule evaluations
        rule_id_mapping = {}
        for ind, rule in rules.iterrows():
            rule_id_mapping[rule['rule_id']] = [rule['rule_string'], rule['next_if_sucess'], rule['next_if_failure'], rule['stage'], rule['description'], rule['data_source']]

        # if columwise processing then run those
        if column:
            output = run_chained_rules_column(file_path, rules)
            return {'flag': True, 'message': 'Applied business rules columnwise successfully.'}

            
        # making it generic takes to take a type parameter from the database..
        # As of now make it (all others  or chained) only
        is_chain_rule = '' not in rule_id_mapping
        
        # get the required table data on which we will be applying business_rules  
        case_id_data_tables = get_data_sources(tenant_id, case_id, 'case_id_based') 
        master_data_tables = get_data_sources(tenant_id, case_id, 'master', master=True)
        
        # consolidate the data into data_tables
        data_tables = {**case_id_data_tables, **master_data_tables} 
        
        logging.info(f"\ndata got from the tables is\n")
        logging.info(data_tables)
        
        updates = {}
        # apply business rules
        if is_chain_rule:
            run_chained_rules(case_id, tenant_id, rules)
        else:
            updates = run_group_rules(case_id, list(rules['rule_string']), data_tables)
            
        # update in the database, the changed fields eventually when all the stage rules were got
        update_tables(case_id, tenant_id, updates)
        
        #  return the updates for viewing
        return {'flag': True, 'message': 'Applied business rules successfully.', 'updates':updates}
    except Exception as e:
        logging.exception('Something went wrong while applying business rules. Check trace.')
        return {'flag': False, 'message': 'Something went wrong while applying business rules. Check logs.', 'error':str(e)}

    
    
    

In [ ]:
# %load BusinessRules.py
import Lib
import _StaticFunctions
import _BooleanReturnFunctions
import _AssignFunction
from datetime import datetime
import pandas as pd

# comment below two for local testing
#from ace_logger import Logging
#logging = Logging()

# uncomment these below lines for local testing
import logging 
logger=logging.getLogger() 
logger.setLevel(logging.ERROR) 



@Lib.add_methods_from(_StaticFunctions, _BooleanReturnFunctions, _AssignFunction) 
class BusinessRules():
    
    def __init__(self, case_id, rules, table_data, decision = False):
        self.case_id = case_id
        self.rules = rules
        self.data_source = table_data
        self.is_decision = decision

        # fields which we are maintaining
        self.changed_fields = {}
        self.params_data = {}
        self.params_data['input'] = []

    def evaluate_business_rules(self):
        """Evaluate all the rules"""
        for rule in self.rules:
            logging.info("\n Evaluating the rule: " +f"{rule} \n")
            output = self.evaluate_rule(rule)
            logging.info("\n Output: " +f"{output} \n")
        # update the changes fields in database
        logging.info(f"\nchanged fields are \n{self.changed_fields}\n")
        return self.changed_fields
    
    def evaluate_rule(self,rule):
        """Evaluate the rule"""
        logging.info(f"\nEvaluating the rule \n{rule}\n")

        rule_type = rule['rule_type']
        
        if  rule_type == 'static':
            function_name = rule['function']
            parameters = rule['parameters']
            return  self.evaluate_static(function_name, parameters)
    
        if rule_type == 'condition':
            evaluations = rule['evaluations']
            return self.evaluate_condition(evaluations)
    
    def conditions_met(self, conditions):
        """Evaluate the conditions and give out the final decisoin
        
        """
        eval_string = ''
        # return True if there are no conditions...that means we are doing else..
        if not conditions:
            return True
        # evaluate the conditions
        for condition in conditions:
            logging.info(f"Evaluting the condition {condition}")
            if condition == 'AND' or condition == 'OR':
                eval_string += ' '+condition.lower()+' '
            else:
                eval_string += ' '+str(self.evaluate_rule(condition))+' '
        logging.info(f"\n eval string is {eval_string} \n output is {eval(eval_string)}")
        return eval(eval_string)

    def evaluate_condition(self, evaluations):
        """Execute the conditional statements.

        Args:
            evaluations(dict) 
        Returns:
            decision(boolean) If its is_decision.
            True If conditions met and it is done with executions.
            False For any other case (scenario).
        """
        for each_if_conditions in evaluations:
            conditions = each_if_conditions['conditions']
            executions = each_if_conditions['executions']
            logging.info(f'\nconditions got are \n{conditions}\n')
            logging.info(f'\nexecutions got are \n{executions}\n')
            decision = self.conditions_met(conditions)
            
            """
            Why this self.is_decision and decision ?
                In decison tree there are only one set of conditions to check
                But other condition rules might have (elif conditions which needs to be evaluate) 
            """
            if self.is_decision:
                if decision:
                    for rule in executions:
                        self.evaluate_rule(rule)
                logging.info(f"\n Decision got for the (for decision tree) condition\n {decision}")    
                return decision
            if decision:
                for rule in executions:
                    self.evaluate_rule(rule)
                return True
        return False

    def get_param_value(self, param_object):
        """Returns the parameter value.

        Args:
            param_object(dict) The param dict from which we will parse and get the value.
        Returns:
            The value of the parameter
        Note:
            It returns a value of type we have defined. 
            If the parameter is itself a rule then it evaluates the rule and returns the value.
        """
        logging.info(f"\nPARAM OBJECT IS {param_object}\n")
        param_source = param_object['source']
        if param_source == 'input_config':
            table_key = param_object['table']
            column_key = param_object['column']
            table_key = table_key.strip() # strip for extra spaces
            column_key = column_key.strip() # strip for extra spaces
            logging.debug(f"\ntable is {table_key} and column key is {column_key}\n")
            try:
                data = {}
                # update params data
                data['type'] = 'from_table'
                data['table'] = table_key
                data['column'] = column_key
                data['value'] = self.data_source[table_key][column_key]
                self.params_data['input'].append(data)
                return data['value']
            except Exception as e:
                logging.error(f"\ntable or column key not found\n")
                logging.error(str(e))
                logging.info(f"\ntable data is {self.data_source}\n")
        if param_source == 'rule':
            param_value = param_object['value']
            return self.evaluate_rule(param_value)
        if param_source == 'input':
            param_value = param_object['value']
#             param_value = str(param_value).strip() # converting into strings..need to check
            return  param_value

##############################################################################################################

read_rule = {'rule_type':'static',
              'function':'Read',
              'parameters':{'path':'D:\\AlgonoX\\Python\\BUSINESS RULES\\KARVY_BusinessRules\\sample.csv'}
}

DR_CR_D_check = {'rule_type': 'static',
        'function': 'CompareKeyValue',
        'parameters': {'left_param':{'source': 'rule', 'value':read_rule},
                       'operator':'==',
                       'right_param':{'source':'input', 'value':'D'}
                      }
       }

Amount_Debit = {'rule_type': 'static',
                'function': 'Transform',
                'parameters':[
            {'param':{'source':'rule', 'value':read_rule}},
            {'operator':'*'},
            {'param':{'source':'input', 'value':"-1"}}
        ]
}


Amount_Assign_rule = {'rule_type': 'static',
                        'function': 'Assign',
                        'parameters': {'assign_table':{'table':'karvy_feed_copy', 'column':'Amount'}, 
                                                'assign_value':{'source':'rule', 'value':Amount_Debit}
                      }
}

updating_Amount = {'rule_type': 'condition',
                    'evaluations': [{ 'conditions':[DR_CR_D_check],'executions':[Amount_Assign_rule]

        }]
}


########################################### Assign_Rules ###############################################################################

Feed_Assign_rule = {'rule_type': 'static',
                        'function': 'Assign',
                        'parameters': {'assign_table':{'table':'karvy_feed_copy', 'column':'Feed'}, 
                                                'assign_value':{'source':'input', 'value':'CMS'}
                      }
}

Sub_Feed_Assign_rule = {'rule_type': 'static',
                        'function': 'Assign',
                        'parameters': {'assign_table':{'table':'karvy_feed_copy', 'column':'Sub_Feed'}, 
                                                'assign_value':{'source':'input', 'value':'CMS'}
                      }
}

ID_Assign_rule = {'rule_type': 'static',
                        'function': 'Assign',
                        'parameters': {'assign_table':{'table':'karvy_feed_copy', 'column':'ID'}, 
                                                'assign_value':{'source':'input', 'value':'0'}
                      }
}

Feed_ID_Assign_rule = {'rule_type': 'static',
                        'function': 'Assign',
                        'parameters': {'assign_table':{'table':'karvy_feed_copy', 'column':'Feed_ID'}, 
                                                'assign_value':{'source':'input', 'value':''}
                      }
}

########################################### Filtering_rule ###################################################################

Code_check = {'rule_type': 'static',
        'function': 'CompareKeyValue',
        'parameters': {'left_param':{'source': 'input_config', 'table': 'karvy_feed_copy', 'column': 'Code'},
                       'operator':'==',
                       'right_param':{'source':'input', 'value':''}
                      }
       }

Queue_Assign_rule = {'rule_type': 'static',
                        'function': 'Assign',
                        'parameters': {'assign_table':{'table':'Process_queue', 'column':'queue'}, 
                                                'assign_value':{'source':'input', 'value':'Maker'}
                      }
}    

updating_Queue_Code = {'rule_type': 'condition',
                    'evaluations': [{ 'conditions':[Code_check],'executions':[Queue_Assign_rule]

        }]
}

########################################### Filtering_Date ###################################################################
Filtering_Date = { 'rule_type': 'static',
    'function'  : 'InCheckDate',
    'parameters':{'start_date':{'source':'input', 'value':'2009-01-01'},
                'end_date':{'source':'input', 'value':'2029-01-01'},
                'date':{'source':'input_config','table': 'karvy_feed_copy', 'column': 'Date'}
                }
}

###################################################################################################################################


db_tables = {
                    "karvy_extraction" : ["karvy_feed_copy"],
                    #"queues":["process_queue"]
                }
unique_id = 'Cms_10'

###################################################################################################################################
###################################### Based on Dataframe #########################################################################
"""data = {
    'NAME':['Rachel','Ross','joy','Monica','Phoebe'],
    'ROLL':[497,498,499,500,501],
    'AGE':[23,24,25,26,27]
}

data = pd.DataFrame(data)"""

####################################################################################################################################
count_df_rule = {'rule_type':'static',
                  'function':'Contains',
                  'parameters':{'table_name':'data','column_name':'NAME','value':{'source':'input','value':'Rachel'}}
}

assign_df_rule = {'rule_type':'static',
                  'function':'Assign',
                  'parameters':{'assign_table':{'table':'data', 'column':'ROLL'}, 
                                'assign_value':{'source':'input', 'value':'roll is'}
}
}
#####################################################################################################################################
############################################# Karvy_BusinessRules #########################################################
#####################################################################################################################################

read_rule = {'rule_type':'static',
              'function':'Read',
              'parameters':{'path':'D:\\AlgonoX\\Python\\BUSINESS RULES\\KARVY_BusinessRules\\sample.csv'}
}

#####################################################################################################################################
filtering_rule = {'rule_type': 'static',
                  'function': 'Filter',
                  'parameters': {
                  'from_table': {'source':'input','value':'ocr'},
                 'lookup_filters':[
                {
                    'column_name': 'Amount',
                    'compare_with':  {'source':'input', 'value':1000},
                    'operator':'&'
                },
                {
                    'column_name': 'Plan Code',
                    'compare_with':  {'source':'input', 'value':'AG'},
                    'operator':'|'
                }
            ]
        }

}
########################################## 05-10-2019 #######################################################################

# ocr = pd.read_csv('D:\\AlgonoX\\Python\\BUSINESS RULES\\KARVY_BusinessRules\\sample.csv')

amount_transform = {'rule_type': 'static',
                'function': 'TransformDF',
                'parameters':{
                                'table':'ocr',
                                'value1_colomn': {'source':'input', 'value': 'Amount'},
                                'operator':'*',
                                'value2': {'source':'input', 'value': -1}
                            }
}

# ignore filter_test rule
filter_test = {'rule_type': 'static',
    'function': 'Filter',
    'parameters': {
            'from_table': 'ocr',
            'lookup_filters':[
                {
                    'column_name': 'Amount',
                    'lookup_operator' : '==',
                    'compare_with':  {'source':'input', 'value': 10}
                }
            ]
        }
}

where_test = {'rule_type': 'static',
    'function': 'WhereClause',
    'parameters': {
            'data_frame':{'source':'rule', 'value': Amount_Debit},
            'from_table': 'ocr',
            'colomn':'Amount',
            'lookup_filters':[
                {
                    'column_name': 'Plan Code',
                    'lookup_operator' : '==',
                    'compare_with':  {'source':'input', 'value': 'GP'}
                }
                
            ]

        }
}


###########################################################################################################
read_rule = {'rule_type':'static',
              'function':'Read',
              'parameters':{
                            'path':{'source':'input','value':'C:\\Users\\Algonox\\Desktop\\AlgonoxWork\\BusinessRulesModule\\run_business_rule\\sample.csv'},
                           'table_name':{'source':'input','value':'ocr'}}
}

#####################################################################################################################################
add_filetered_df_rule = {'rule_type':'static',
                  'function':'Assign',
                  'parameters':{'assign_table':{'table':'ocr', 'column':'Filtered'}, 
                                'assign_value':{'source':'input', 'value':'N'}
                }
             }

#################################################################################################################################
add_matched_assign_df_rule = {'rule_type':'static',
                  'function':'Assign',
                  'parameters':{'assign_table':{'table':'ocr', 'column':'Matched'}, 
                                'assign_value':{'source':'input', 'value':'0'}
                }
             }
########################################################################################################################
add_dr_cr_assign_df_rule = {'rule_type':'static',
                  'function':'Assign',
                  'parameters':{'assign_table':{'table':'ocr', 'column':'DR_CR'}, 
                                'assign_value':{'source':'input', 'value':'D'}
                }
             }
#######################################################################################################################
add_amount = {'rule_type':'static',
                  'function':'Assign',
                  'parameters':{'assign_table':{'table':'ocr', 'column':'Amount'}, 
                                'assign_value':{'source':'input', 'value':50}
                }
             }
#######################################################################################################################

amount_transform = {'rule_type': 'static',
                'function': 'TransformDF',
                'parameters':{
                                'table':'ocr',
                                'value1_colomn': {'source':'input', 'value': 'Amount'},
                                'operator':'*',
                                'value2': {'source':'input', 'value': -1}
                            }
}

# assign_amoutn_transform =  {'rule_type':'static',
#                   'function':'Assign',
#                   'parameters':{'assign_table':{'table':'ocr', 'column':'Amount'}, 
#                                 'assign_value':{'source':'rule', 'value':amount_transform}
#                 }
#              }

where_rule = {'rule_type': 'static',
    'function': 'WhereClause',
    'parameters': {
            'data_frame':{'source':'rule', 'value': amount_transform},
            'from_table': 'ocr',
            'column':'Amount',
            'lookup_filters':[
                {
                    'column_name': 'DR_CR',
                    'lookup_operator' : '==',
                    'compare_with':  {'source':'input', 'value': 'D'}
                }
                
            ]

        }
}


assign_where_transform =  {'rule_type':'static',
                  'function':'Assign',
                  'parameters':{'assign_table':{'table':'ocr', 'column':'Amount'}, 
                                'assign_value':{'source':'rule', 'value':where_rule}
                }
             }

rules = [read_rule, add_amount, add_filetered_df_rule, add_matched_assign_df_rule, add_dr_cr_assign_df_rule, assign_where_transform]
rules= [read_rule]
a = BusinessRules('1234',rules,{},decision = True)
(a.evaluate_business_rules())



In [ ]:
def writeToCsv(df, file_path, required_standard_mapping):
    """Write the dataframe to csv"""
    try:

        # first find whether any csv exists with the file_name
        existing_df = pd.read_csv(file_path)

        mode = 'a'
        headers = False
        df.to_csv(file_path, columns=required_columns, mode=mode, header=headers, index=False)
    except:
        # no sample file exists
        headers = True
        mode = 'w'
        
        # rename the columns in the dataframe to standardized columns            
        df = df.rename(columns=required_standard_mapping)
        
        required_columns = required_standard_mapping.values()

        df.to_csv(file_path, columns=required_columns, mode=mode, header=headers, index=False)
                
    return "Written to csv successfully"

In [4]:
import pandas as pd
sample = pd.read_csv('sample.csv')

C:\Users\Algonox\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (42,48,51,65) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
list(sample.columns)
s_req_ = {'Investor type' : 'TYPE'}
file_path = 'C:\\Users\\Algonox\\Desktop\\sample_small.csv'
df = sample.head(5)
writeToCsv(df, file_path, s_req_)
# df.re

In [ ]:
sample.head(5).to_csv('C:\\Users\\Algonox\\Desktop\\sample_small.csv')

In [ ]:
a = [1,2,3]
b = [1,2,3]

c = {e:f for (e,f) in zip(a,b)}
c

In [ ]:
sample.head(5)
sample.head(5).to_csv('C:\\Users\\Algonox\\Desktop\\sample_small_.csv', header=False)

In [ ]:
b = a.data_source['ocr'].head(5)

In [ ]:
b['Investor type']

In [2]:
sample.head(5)

NameError: name 'sample' is not defined

In [ ]:
a = "12345"

start = -2
end = None
a[start:end]

In [13]:
import numpy as np

In [14]:
sample['Amount'].iloc[0] = np.nan

C:\Users\Algonox\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
sample['Amount']*-1

0            NaN
1        -150000
2           -500
3          -2000
4          -2000
5         -40000
6         -39000
7         -50000
8         -50000
9        -190000
10         -1000
11         -1000
12       -170000
13        -1e+06
14         -2500
15        -25000
16        -50000
17         -1500
18         -3000
19         -4000
20         -1500
21         -5000
22         -1000
23         -1000
24        -15000
25        -15000
26         -1550
27         -1000
28         -5000
29         -1000
           ...  
166701     -1000
166702     -2000
166703    -25000
166704     -2000
166705     -5000
166706     -2000
166707    -25000
166708     -2000
166709     -2000
166710     -5000
166711     -5000
166712     -5000
166713     -2000
166714     -5900
166715     -5000
166716    -14000
166717     -2000
166718    -50000
166719     -2000
166720     -2900
166721    -25000
166722     -2000
166723     -2000
166724     -2000
166725     -2000
166726     -2000
166727      -100
166728    -200